# Binary classificiation and transfer learning with EfficientNetB0

By Evelyn Fitzgerald (acf67)

In this notebook, I show that an EfficientNetB0 NN pre-trained on the ImageNet dataset can be used for a binary classification task. The EfficientNet family of NNs is great because the NNs achieve better performance for a lower computational and memory overhead than other state-of-the-art image classifiers \[[1][enet]]. So whereas the original DeepSolar NNs were based on Inception v3, I'd like to use the EfficientNets.

One problem I had was that while EfficientNetB0 could be loaded into Keras with pre-trained weights, the number of classes had to be 1000 (the number of classes in ImageNet). So instead of using the NN as is, I had to get the pre-trained NN without the final layer by setting `include_top=False`, then attaching my own two-class output layer to the end. The pre-trained layers have to be frozen so they won't be corrupted during fine-tuning (which I don't do in this notebook... yet).

I also added a `GlobalAveragePooling2D` layer to merge all of the pixels in each channel into a single data point, so that the input to the final layer has a fixed size regardless of the size of the input image.

`BatchNormalization` is also included in the tutorial so I copied it in but I don't actually know why it's needed. I think it's feature scaling?

[enet]: https://github.com/tensorflow/tpu/tree/master/models/official/efficientnet

Reference: [Image classification via fine-tuning with EfficientNet](https://keras.io/examples/vision/image_classification_efficientnet_fine_tuning/)

In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
img0 = tf.random.uniform(shape=(1,224,224,3))

In [3]:
enet0_seg = keras.applications.EfficientNetB0(include_top=False)

In [4]:
# Freeze this layer so it won't be corrupted during fine-tuning
enet0_seg.trainable = False

In [5]:
out1 = enet0_seg.predict(img0)

In [6]:
out1.shape

(1, 7, 7, 1280)

In [7]:
enet0_binary = keras.Sequential(
    [
        enet0_seg,
        keras.layers.GlobalAveragePooling2D(name="avg_pool"),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(2, activation="softmax", name="classify")
    ],
    name="efficientnetb0_binary"
)

In [8]:
enet0_binary.summary()

Model: "efficientnetb0_binary"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb0 (Functional)  (None, None, None, 1280)  4049571   
_________________________________________________________________
avg_pool (GlobalAveragePooli (None, 1280)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 1280)              5120      
_________________________________________________________________
classify (Dense)             (None, 2)                 2562      
Total params: 4,057,253
Trainable params: 5,122
Non-trainable params: 4,052,131
_________________________________________________________________


In [9]:
# Tell us how long it takes to classify a single image
from time import time

start_time = time()

out2 = enet0_binary.predict(img0)

elapsed = time() - start_time
print(f"Time: {elapsed:.5f} s")

Time: 0.81268 s


In [10]:
out2.shape

(1, 2)

In [11]:
out2

array([[0.5021458, 0.4978542]], dtype=float32)